In [2]:
from pprint import pprint
from SimpleEarnAPY import main, formatDataFrame
from FetchBinance import Binance
assets = ['USDC', 'USDT']

In [ ]:
binance = Binance(assets=assets)
rate = binance.simpleEarn('USDT')
rate = binance._GetCleanResult(rate)
pprint(rate)

In [ ]:
rate = binance.SimpleEarnRates
rate
df = formatDataFrame(binance.__class__.__name__, rate)
df

In [ ]:
for key, value in rate.items():
    print(key)
    print(value)

In [21]:
import pandas as pd

for asset in assets:
    df = pd.DataFrame.from_dict(rate[asset])
    df['coin'] = [asset]*len(df)
    df['cex'] = [binance.__class__.__name__]*len(df)
    df = df[['cex','coin','rate','amt']]

In [4]:
from SimpleEarnAPY import main
from pprint import pprint
df = main()

In [ ]:
df

In [ ]:
print(df.to_string(index=False))

In [1]:
from FetchBybit import Bybit
assets = ['USDC', 'USDT']
bybit = Bybit(assets=assets)
bybit.getSimpleEarnRates()

Finish fetching USDC
Multithreaded execution took 10.11 seconds.

Finish fetching USDT
Multithreaded execution took 3.01 seconds.



In [2]:
bybit.SimpleEarnRates

{'USDC': [{'amt': '<= 500 USDC', 'rate': 10.47},
  {'amt': '> 500 USDC', 'rate': 5.47}],
 'USDT': [{'amt': '<= 500 USDT', 'rate': 8.95},
  {'amt': '> 500 USDT', 'rate': 3.95}]}